# Cart Pole

#### Descripcion 


+ **Espacio de observacion (Box(4,))**

+ Posicion del carro: [-2.4, 2.4]
+ Velocidad del carro: [$-\infty$, $\infty$]
+ Angulo del palo: [-41.8, 41.8]
+ Velocidad del palo en la punta: [$-\infty$, $\infty$]


+ **Espacio de accion (Discrete(2))**

+ Izquierda: 0
+ Derecha: 1


El objetivo es mantener el palo vertical moviendo a izquierda y derecha el carro.


La recompensa es +1 para cada paso temporal. El episodio se termina si el angulo es mayor a $\pm 12$ grados o si el carro sobrepasa la posicion $\pm 2.4$

**Ahora usaremos Keras para crear el agente**

In [1]:
# pip3 install keras-rl

In [3]:
# librerias

import gym
import numpy as np
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import EpsGreedyQPolicy

import warnings
warnings.simplefilter('ignore')

### Agente

En este vamos caso a crear un agente basado en el algoritmo SARSA (state-action-reward-state-action). Dicho algoritmo tiene la ecuacion:

$$Q_{nueva}(e_{t}, a_{t})=(1-\alpha)·Q(e_{t}, a_{t}) + \alpha·[r_{t} + \gamma·Q(e_{t+1}, a_{t+1})]$$


donde:
+ $e_{t}$ es el estado en el tiempo t
+ $a_{t}$ es la accion en el tiempo t
+ $\alpha$ es la tasa de aprendizaje $(0<\alpha \leq{1})$
+ $Q(e_{t}, a_{t})$ es el viejo valor de calidad
+ $[r_{t} + \gamma·Q(e_{t+1}, a_{t+1})]$ es el valor aprendido
+ $r_{t}$ es la recompensa recibida al pasar del estado $e_{t}$ al estado $e_{t+1}$
+ $\gamma$ es el factor de descuento $(0\leq \gamma \leq 1)$. Evalua las recompensas recibidas anteriormente con un valor mayor que las recibidas posteriormente, se puede interpretar como la probabilidad de tener exito (o sobrevivir) en cada paso temporal+ $\max_{a}Q(e_{t+1}, a_{t+1})$ es la estimacion del valor optimo futuro

In [4]:
class Agente(object):
    
    def __init__(self, entorno):
        self.entorno=entorno
        self.observaciones=self.entorno.observation_space.shape[0]
        self.dim_accion=self.entorno.action_space.n
        self.pasos=NUM_MAX_PASOS
        self.episodios=NUM_MAX_EPIS
        self.politica=EpsGreedyQPolicy()
        
        
    def sarsa(self, entrenar=True, guardar=False, cargar=False):  # modelo sarsa con keras
        modelo=Sequential()
        modelo.add(Flatten(input_shape = (1, self.observaciones)))
        modelo.add(Dense(24, activation='relu'))
        modelo.add(Dense(24, activation='relu'))
        modelo.add(Dense(24, activation='relu'))
        modelo.add(Dense(self.dim_accion, activation='linear'))
        
        modelo=SARSAAgent(model=modelo, policy=self.politica, nb_actions=self.dim_accion)
        
        modelo.compile('adam', metrics=['mse'])
        
        
        if entrenar:
            modelo.fit(self.entorno, nb_steps=self.pasos, visualize=False, verbose=1)
        
        if guardar:
            modelo.save_weights('sarsa_weights.h5f', overwrite=True)
            
        if cargar:
            modelo.load_weights('sarsa_weights.h5f')
            
        return modelo

    
    
    def juega(self, ver):
        return self.aprende.test(self.entorno, nb_episodes=self.episodios, visualize=ver)

In [5]:
# constantes

NUM_MAX_PASOS=50000
NUM_MAX_EPIS=100

In [6]:
entorno=gym.make('CartPole-v1')

In [ ]:
agente=Agente(entorno).sarsa()





Training for 50000 steps ...
Interval 1 (0 steps performed)


 2166/10000 [=====>........................] - ETA: 12s - reward: 1.0000

In [ ]:
stats=agente.juega(False)
print('Recompensa media en 100 episodios:{}'.format(np.mean(scores.history['episode_reward'])))

In [ ]:
agente.juega(True)